Lets build our NIC to ontology map from what's already been submitted.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *

In [3]:
# live server & control file
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [5]:
#search = "/search/?type=Biosample&lab.title=Barbara+Wold%2C+Caltech&award.rfa=ENCODE4"
#search = "/search/?type=Experiments&lab.title=Barbara+Wold%2C+Caltech&award.rfa=ENCODE4"

In [6]:
graph = server.search_jsonld(**{
    'type': 'Experiment', 
    'lab.title': "Barbara Wold, Caltech",
    'award.rfa': "ENCODE4",
}, limit='all')

In [7]:
ontology_nic_map = {}
ontology_biosample_map = {}
ontology_term_name = {}
biosamples_missing_nic = {}
for slim in graph['@graph']:
    for replicate in slim['replicates']:
        library = replicate['library']
        if 'biosample' in library:
            biosample = library['biosample']
            biosample_id = '/biosamples/{}/'.format(biosample['accession'])
            biosample = server.get_json(biosample_id)
            lab = biosample['lab']
            lab_id = lab['@id']
            ontology = biosample['biosample_ontology']
            ontology_id = ontology['@id']
            nic = biosample.get("nih_institutional_certification")
            if nic is not None:
                ontology_nic_map.setdefault(ontology_id, set()).add(nic)
                ontology_biosample_map.setdefault(ontology_id, set()).add((biosample_id, (nic, lab_id)))
                ontology_term_name[ontology_id] = ontology["term_name"]
            else:
                biosamples_missing_nic.setdefault(ontology_id, set()).add(biosample_id)


From https://docs.google.com/document/d/1UmxQMg2xDci6bR8p-_XxFesoBMXWdTmPynaQrHZIbZA/edit?skip_itp2_check=true

In [8]:
our_nics = [
    'NIC00004',
    'NIC00005',
    'NIC00011',
    'NIC00015',
    'NIC00022',
    'NIC00023',
    'NIC00024',
    'NIC00025',
    'NIC00026',
    'NIC00027',
    'NIC00044',
]

In [9]:
rows = []
for ontology in ontology_nic_map:
    term_name = ontology_term_name[ontology]
    for nic in ontology_nic_map[ontology]:
        if nic in our_nics:
            rows.append((ontology, term_name, nic))
df = pandas.DataFrame(rows, columns=['ontology', 'term_name', 'nic']).set_index('ontology')
df.sort_values('nic')

,term_name,nic
ontology,,
/biosample-types/tissue_UBERON_0008971/,left colon,NIC00005
/biosample-types/tissue_UBERON_0001115/,left lobe of liver,NIC00005
/biosample-types/tissue_UBERON_0001072/,posterior vena cava,NIC00005
/biosample-types/tissue_UBERON_0002084/,heart left ventricle,NIC00005
/biosample-types/tissue_UBERON_0002113/,kidney,NIC00005
/biosample-types/tissue_NTR_0000493/,left ventricle myocardium inferior,NIC00005
/biosample-types/tissue_UBERON_0008450/,psoas muscle,NIC00005
/biosample-types/tissue_UBERON_0000947/,aorta,NIC00005
/biosample-types/tissue_UBERON_0015143/,mesenteric fat pad,NIC00005


In [11]:
df.sort_values('nic').to_csv("biosample_nic.csv")